### Example notebook to analyze remote data on Zeus - C-GLORSv7

#### Prerequisite: prepare the environment 

1. Connect to the Zeus VPN using the VPN client, e.g. TunnelBlick

2. Add your local public key in the .ssh/authorized_keys on your Zeus account

3. Create the full conda environment starting from the requirements listed in *environment.yml* by running the following commands:

`conda env create -f environment.yml`

`source activate zapata`

Import Python utilities and setup Zeus username (replace YOURUSERNAME with your account)

In [16]:
import os, sys
sys.path.append("../")
import zeus
zeus.init(user="tl28319",host='zeus')

Start a Dask client on Zeus with 1 worker node (*n_workers = 1*)

In [17]:
from distributed import Client, get_client
client = zeus.start_dask(name="Zapata",
              project="R000",
              cores=36,
              processes=12,
              memory="50 GB",
              interface="ib0",
              walltime="00:30", 
              job_extra=["-x"], 
              n_workers=1
             )

/users_home/oda/tl28319/dask-space/connection
Scheduler address is: tcp://192.168.118.12:44308
Job <21369> is submitted to queue <p_short>.



Check if it the cluster has been allocated on LSF (some time is required before LSF provides the resources; try to run the cell again after a few seconds). When available click on the **Dashboard** link to open the Dask monitoring interface.

In [18]:
client

Client Scheduler: tcp://192.168.118.12:44308 Dashboard: http://192.168.118.12:39398/status,Cluster Workers: 12 Cores: 36 Memory: 50.00 GB


Let's load the C-GLORSv7 dataset in parallel using Xarray using zapata data interface.

In [19]:
def read_remote():
    import sys
    sys.path.append('/users_home/oda/tl28319/GIT/Zapata')                       
    import zapata.data as zdat
    client = get_client()
    thedata = zdat.read_xarray(dataset='C-GLORSv7', var='votemper', period=[2000, 2001], season='JFM', level=[500.])
    return thedata

future = client.submit(read_remote)
temp = future.result()

Show the computation result

In [23]:
print(temp)
print("Size in bytes is : %i" % temp.nbytes)

<xarray.DataArray (time: 2, lev: 1, lat: 1050, lon: 1442)>
dask.array<getitem, shape=(2, 1, 1050, 1442), dtype=float64, chunksize=(1, 1, 1050, 1442), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2000-03-31 2001-03-31
  * lev      (lev) float32 508.6399
Dimensions without coordinates: lat, lon
Attributes:
    standard_name:       sea_water_potential_temperature
    long_name:           temperature
    units:               degC
    online_operation:    average
    interval_operation:  1080 s
    interval_write:      1 d
    cell_methods:        time: mean (interval: 1080 s)
    coordinates:         time_centered deptht gphit glamt
    realm:               ocn
    time_resample:       JFM
Size in bytes is : 24225600


Create a map of the average temperature using zapata mapping tools.

In [ ]:
import zapata.mapping as zmap

Shutdown the cluster and release the resources on LSF. Note sometimes warnings might be raised by the function due to synchronization effects.

In [24]:
zeus.stop_dask(client)

Job <21369> is being terminated


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
